In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
import operator

import os
os.environ['PYSPARK_PYTHON'] = '/usr/local/anaconda3/envs/spark/bin/python3'

In [2]:
sc = pyspark.SparkContext(appName="Matrix Multiplication")

In [3]:
matrix_a_raw = sc.textFile("matrixA.txt")

In [4]:
matrix_b_raw = sc.textFile("matrixB.txt")

In [5]:
spark = SparkSession(sc)

In [6]:
def to_matrix_a(x):
    i, j, v = x.split()
    return (j, (i, v))

def to_matrix_b(x):
    j, k, v = x.split()
    return (j, (k, v))

In [7]:
def to_matrix_entry(x):
    i, j, v = x.split()
    return MatrixEntry(i, j, v)

In [8]:
entries_a = matrix_a_raw.map(to_matrix_a)

In [9]:
entries_b = matrix_b_raw.map(to_matrix_b)

In [10]:
entries_a.collect()

[('0', ('0', '1')),
 ('1', ('0', '2')),
 ('2', ('0', '3')),
 ('0', ('1', '4')),
 ('1', ('1', '5')),
 ('2', ('1', '6'))]

In [11]:
entries_b.collect()

[('0', ('0', '6')),
 ('0', ('1', '3')),
 ('1', ('0', '5')),
 ('1', ('1', '2')),
 ('2', ('0', '4')),
 ('2', ('1', '1'))]

In [12]:
# mat_a = CoordinateMatrix(entries_a)

In [13]:
# mat_b = CoordinateMatrix(entries_b)

In [14]:
# a = mat_a.entries
# a.collect()

In [15]:
def multiply_mat(x):
    left = x[1][0]
    right = x[1][1]
    i, v = left
    k, w = right
    return ((i, k), (int(v) * int(w)))

In [16]:
product_entries = entries_a \
    .join(entries_b) \
    .map(multiply_mat) \
    .reduceByKey(operator.add) \
    .map(lambda x:  (x[0][0], x[0][1], x[1]))

In [17]:
product_entries.collect()

[('0', '0', 28), ('0', '1', 10), ('1', '0', 73), ('1', '1', 28)]